<a href="https://colab.research.google.com/github/marcool5/Cyber/blob/main/Marcool_Cyber_Dataset_ML_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
# data cleaning and plots
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
plt.style.use('ggplot')
%matplotlib inline

# sklearn: data preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

# sklearn: train model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, classification_report

# sklearn classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
# Load data
data = pd.read_csv('/content/drive/MyDrive/dataset/UNSW-NB15_1.csv')
# Look at the first 5 rows
data.head(n=5)
# Print the columns type in order to review the data type for each column of the dataset.
data.dtypes
#Print the total registers for each column.
data.info()
data.value_counts()








<ipython-input-29-1ddd2a30c49f>:30: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/dataset/UNSW-NB15_1.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 49 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   59.166.0.0     700000 non-null  object 
 1   1390           700000 non-null  object 
 2   149.171.126.6  700000 non-null  object 
 3   53             700000 non-null  object 
 4   udp            700000 non-null  object 
 5   CON            700000 non-null  object 
 6   0.001055       700000 non-null  float64
 7   132            700000 non-null  int64  
 8   164            700000 non-null  int64  
 9   31             700000 non-null  int64  
 10  29             700000 non-null  int64  
 11  0              700000 non-null  int64  
 12  0.1            700000 non-null  int64  
 13  dns            700000 non-null  object 
 14  500473.9375    700000 non-null  float64
 15  621800.9375    700000 non-null  float64
 16  2              700000 non-null  int64  
 17  2.1            700000 non-nul

59.166.0.0    1390   149.171.126.6   53   udp   CON  0.001055  132   164     31   29   0  0.1  dns   500473.9375   621800.9375   2   2.1  0.2  0.3  0.4         0.5         66   82    0.6  0.7  0.8          0.9         1421927414  1421927414.1  0.017       0.013       0.10      0.11      0.12      0.13  0.14  0.15  0.16  0.17  3   7   1  3.1  1.1  1.2  1.3  Unnamed: 47     0.18
175.45.176.3  0      149.171.126.17  0    sep   INT  0.000010  104   0       254  0    0  0    -     4.160000e+07  0.000000e+00  2   0    0    0    0           0           52   0     0    0    0.000000     0.000000    1421928685  1421928685    0.010000    0.000000    0.000000  0.000000  0.000000  0     2     0     0     0     12  12  5  4    2    2    4    DoS             1       14
                                          ospf  INT  9.633130  2688  0       254  0    0  0    -     2.179146e+03  0.000000e+00  42  0    0    0    0           0           64   0     0    0    267.166625   0.000000    1421930972  1421

In [38]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load the UNSW-NB15 training dataset
train_data = pd.read_csv('/content/drive/MyDrive/dataset/UNSW_NB15_testing-set.csv')

# Data preprocessing: Drop unnecessary columns and handle missing values (similar to the testing data preprocessing)
train_data = train_data.drop(columns=['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'service', 'ct_src_ltm', 'ct_dst_ltm', 'attack_cat'])
train_data = train_data.fillna(0)

# Separate features (X_train) and target labels (y_train)
X_train = train_data.drop(columns=['label'])
y_train = train_data['label']

# Create and train a Random Forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Save the trained model to a file using joblib
model_filename = 'random_forest_model.pkl'
joblib.dump(clf, model_filename)

print(f"Pre-trained model saved as '{model_filename}'.")

Pre-trained model saved as 'random_forest_model.pkl'.


In [41]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import joblib

# Load the UNSW-NB15 testing dataset
test_data = pd.read_csv('/content/drive/MyDrive/dataset/UNSW_NB15_testing-set.csv')

# Data preprocessing: Drop unnecessary columns and handle missing values (similar to the training data preprocessing)
test_data = test_data.drop(columns=['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'service', 'ct_src_ltm', 'ct_dst_ltm', 'attack_cat'])
test_data = test_data.fillna(0)

# Separate features (X_test) and target labels (y_test)
X_test = test_data.drop(columns=['label'])
y_test = test_data['label']

# Load the pre-trained model (e.g., Random Forest classifier)
clf = joblib.load('random_forest_model.pkl')  # Replace 'your_pretrained_model.pkl' with the actual model file

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Create the confusion matrix
confusion = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion)

# Print a classification report for more detailed metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[ 56000      0]
 [     0 119341]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56000
           1       1.00      1.00      1.00    119341

    accuracy                           1.00    175341
   macro avg       1.00      1.00      1.00    175341
weighted avg       1.00      1.00      1.00    175341

